In [ ]:
### Uncomment the following lines to install the required packages###
# # For Microsoft Entra ID token authentication support 
# ! pip install azure-identity 
# # For Azure OpenAI Assistant 
# ! pip install "autogen-ext[openai]"
# # This module provides the main entry point for the autogen_agentchat package 
# ! pip install autogen-agentchat
# # For Azure AI Projects client library from Azure AI Foundry SDK packages and services
# ! pip install azure-ai-projects

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool
import os

In [ ]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [ ]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="<your deployment model name>",  # e.g., "gpt-4o"
    api_version="<model version>",  # e.g., "2023-05-15"
    model = "<your deployment model name>",  # e.g., "gpt-4o"
    azure_endpoint="<Azure OpenAI Service endpoint>",
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key="<Your API Key from AI foundry>", # For key-based authentication.
)

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='<Your Project connection string from Azure AI Foundry>',
)

In [ ]:
bing_connection = project_client.connections.get(
    connection_name='<Your bing connection name>',
)
conn_id = bing_connection.id

In [ ]:
# The stock price trends tool
async def stock_price_trends_tool(stock_name: str) -> str:
    """
    A dedicated Bing call focusing on real-time stock prices,
    changes over the last few months for 'stock_name'.
    """
    print(f"[stock_price_trends_tool] Fetching stock price trends for {stock_name}...")
    bing = BingGroundingTool(connection_id=conn_id)
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="stock_price_trends_tool_agent",
        instructions=(
            f"Focus on retrieving real-time stock prices, changes over the last few months, "
            f"and summarize market trends for {stock_name}."
        ),
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )

    # Create a new thread and send the user query
    thread = project_client.agents.create_thread()
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=f"Please get stock price trends data for {stock_name}."
    )
    # Process the run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    messages = project_client.agents.list_messages(thread_id=thread.id)

    # Clean up
    project_client.agents.delete_agent(agent.id)

    # Return the Bing result
    return messages["data"][0]["content"][0]["text"]["value"]

In [ ]:
### The news analysis tool.
async def news_analysis_tool(stock_name: str) -> str:
    """
    A dedicated Bing call focusing on the latest news for 'stock_name'.
    """
    print(f"[news_analysis_tool] Fetching news for {stock_name}...")
    bing = BingGroundingTool(connection_id=conn_id)
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="news_analysis_tool_agent",
        instructions=f"Focus on the latest news highlights for the stock {stock_name}.",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )

    thread = project_client.agents.create_thread()
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=f"Retrieve the latest news articles and summaries about {stock_name}."
    )
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    messages = project_client.agents.list_messages(thread_id=thread.id)

    # Clean up
    project_client.agents.delete_agent(agent.id)

    return messages["data"][0]["content"][0]["text"]["value"]


In [ ]:
### The news sentiment analysis tool.
async def market_sentiment_tool(stock_name: str) -> str:
    """
    A dedicated Bing call focusing on overall market sentiment
    for 'stock_name'.
    """
    print(f"[market_sentiment_tool] Fetching sentiment for {stock_name}...")
    bing = BingGroundingTool(connection_id=conn_id)
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="market_sentiment_tool_agent",
        instructions=(
            f"Focus on analyzing general market sentiment regarding {stock_name}."
        ),
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )

    thread = project_client.agents.create_thread()
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=(
            f"Gather market sentiment, user opinions, and overall feeling about {stock_name}."
        )
    )
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    messages = project_client.agents.list_messages(thread_id=thread.id)

    # Clean up
    project_client.agents.delete_agent(agent.id)

    return messages["data"][0]["content"][0]["text"]["value"]

In [ ]:
## The analyst reports tool.
async def analyst_reports_tool(stock_name: str) -> str:
    """
    A dedicated Bing call focusing on analyst reports
    for 'stock_name'.
    """
    print(f"[analyst_reports_tool] Fetching analyst reports for {stock_name}...")
    bing = BingGroundingTool(connection_id=conn_id)
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="analyst_reports_tool_agent",
        instructions=(
            f"Focus on any relevant analyst reports or professional analyses about {stock_name}."
        ),
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )

    thread = project_client.agents.create_thread()
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=(f"Find recent analyst reports, price targets, or professional opinions on {stock_name}.")
    )
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    messages = project_client.agents.list_messages(thread_id=thread.id)

    # Clean up
    project_client.agents.delete_agent(agent.id)

    return messages["data"][0]["content"][0]["text"]["value"]

In [ ]:
# The expert opinions tool.
async def expert_opinions_tool(stock_name: str) -> str:
    """
    A dedicated Bing call focusing on expert or industry leaders' opinions
    for 'stock_name'.
    """
    print(f"[expert_opinions_tool] Fetching expert opinions for {stock_name}...")
    bing = BingGroundingTool(connection_id=conn_id)
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="expert_opinions_tool_agent",
        instructions=(
            f"Focus on industry expert or thought leader opinions regarding {stock_name}."
        ),
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )

    thread = project_client.agents.create_thread()
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=(f"Collect expert opinions or quotes about {stock_name}.")
    )
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    messages = project_client.agents.list_messages(thread_id=thread.id)

    # Clean up
    project_client.agents.delete_agent(agent.id)

    return messages["data"][0]["content"][0]["text"]["value"]


In [ ]:
# -- Trend Data
async def stock_price_trends_agent(stock_name: str) -> str:
    """Agent function for 'stock trends', calls stock_price_trends_tool."""
    return await stock_price_trends_tool(stock_name)

# -- News
async def news_analysis_agent(stock_name: str) -> str:
    """Agent function for 'latest news', calls news_analysis_tool."""
    return await news_analysis_tool(stock_name)

# -- Market Sentiment
async def market_sentiment_agent(stock_name: str) -> str:
    """Agent function for 'market sentiment', calls market_sentiment_tool."""
    return await market_sentiment_tool(stock_name)

# -- Analyst Reports
async def analyst_reports_agent(stock_name: str) -> str:
    """Agent function for 'analyst reports', calls analyst_reports_tool."""
    return await analyst_reports_tool(stock_name)

# -- Expert Opinions
async def expert_opinions_agent(stock_name: str) -> str:
    """Agent function for 'expert opinions', calls expert_opinions_tool."""
    return await expert_opinions_tool(stock_name)

In [ ]:

# -- Stock Price Trends Agent
stock_trends_agent_assistant = AssistantAgent(
    name="stock_trends_agent",
    model_client=az_model_client,
    tools=[stock_price_trends_agent],
    system_message=(
        "You are the Stock Price Trends Agent. "
        "You fetch and summarize stock prices, changes over the last few months, and general market trends. "
        "Do NOT provide any final investment decision."
    )
)
# -- News Analysis Agent
news_agent_assistant = AssistantAgent(
    name="news_agent",
    model_client=az_model_client,
    tools=[news_analysis_agent],
    system_message=(
        "You are the News Agent. "
        "You retrieve and summarize the latest news stories related to the given stock. "
        "Do NOT provide any final investment decision."
    )
)
# -- Market Sentiment Agent
sentiment_agent_assistant = AssistantAgent(
    name="sentiment_agent",
    model_client=az_model_client,
    tools=[
        market_sentiment_agent,
        analyst_reports_agent,
        expert_opinions_agent
    ],
    system_message=(
        "You are the Market Sentiment Agent. "
        "You gather overall market sentiment, relevant analyst reports, and expert opinions. "
        "Do NOT provide any final investment decision."
    )
)
#   -- Investment Decision Agent
decision_agent_assistant = AssistantAgent(
    name="decision_agent",
    model_client=az_model_client,
    # The final agent typically calls the 'investment_decision_agent' to
    # synthesize all the data. If you want it to call the other tools directly,
    # you can also add them here. But typically we rely on the round-robin approach.
    # tools=[investment_decision_agent],
    system_message=(
        "You are the Decision Agent. After reviewing the stock data, news, sentiment, analyst reports, "
        "and expert opinions from the other agents, you provide the final investment decision. In the final decision make a call to either Invest or Not. Also providethe current stock price. "
        "End your response with 'Decision Made' once you finalize the decision."
    )
)

In [ ]:
# Termination condition: stop when the decision agent mentions "Decision Made"
text_termination = TextMentionTermination("Decision Made")
# Max message termination: stop after 15 messages
max_message_termination = MaxMessageTermination(15)
termination = text_termination | max_message_termination

# Round-robin chat among the four agents
investment_team = RoundRobinGroupChat(
    [
        stock_trends_agent_assistant,
        news_agent_assistant,
        sentiment_agent_assistant,
        decision_agent_assistant,
    ],
    termination_condition=termination
)

In [ ]:

async def main():
    stock_name = "MSFT"
    await Console(
        investment_team.run_stream(
            task=f"Analyze stock trends, news, and sentiment for {stock_name}, plus analyst reports and expert opinions, and then decide whether to invest."
        )
    )

In [ ]:
await main()